In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("../input/insurance/train_data.csv")
test = pd.read_csv("../input/insurance/test_data.csv")

In [ ]:
train.shape

In [ ]:
train.head(16)

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.columns

In [ ]:
train['Customer Id'].nunique()

In [ ]:
train.Geo_Code.nunique()

In [ ]:
#What is the relationship between Insured_Period and the Building_Type
train.plot.scatter(x='Insured_Period', y = 'Building_Type', legend = False)

In [ ]:
train.plot.scatter(x='Building Dimension', y = 'Building_Type', legend = False)

In [ ]:
ax = train.groupby('Building_Type').mean().Insured_Period.plot(kind = 'bar', figsize = [5,5])
ax.set_ylabel('Avg. Insured Period')
plt.tight_layout()
plt.show()

In [ ]:
ax = train.groupby('Building_Type').mean().YearOfObservation.plot(kind = 'bar', figsize = [5,5])
ax.set_ylabel('Avg. Yr. of Observation')
plt.tight_layout()
plt.show()
           

In [ ]:
ax = train.Building_Type.hist()
ax.set_xlabel('Building_Type')
ax.set_ylabel('count')

plt.show()

In [ ]:
corr = train.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, vmin=-1, vmax=1, cmap="RdBu")

In [ ]:
sns.heatmap(train.isnull(), yticklabels = False, cbar= False, cmap = 'viridis')

In [ ]:
sns.heatmap(test.isnull(), yticklabels = False, cbar= False, cmap = 'viridis')

In [ ]:
test.isna().sum()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:

train.Building_Painted.replace(to_replace=dict(N='P', V='NP'), inplace=True)
train.Building_Fenced.replace(to_replace=dict(N='F', V='NF'), inplace=True)

In [ ]:
test.Building_Painted.replace(to_replace=dict(N='P', V='NP'), inplace=True)
test.Building_Fenced.replace(to_replace=dict(N='F', V='NF'), inplace=True)

In [ ]:
train['Building_Painted'].head()

In [ ]:
train['Building_Fenced'].head()

In [ ]:
test['Building_Painted'].head()

In [ ]:
test['Building_Fenced'].head()

In [ ]:
mymap = {'   .':0, '>=10':10}

train = train.applymap(lambda s: mymap.get(s) if s in mymap else s)

In [ ]:
mymap = {'   .':0, '>=10':10}

test = test.applymap(lambda s: mymap.get(s) if s in mymap else s)

In [ ]:
test["NumberOfWindows"] = test["NumberOfWindows"].astype(int)

In [ ]:
train["NumberOfWindows"] = train["NumberOfWindows"].astype(int)

In [ ]:
train.NumberOfWindows.unique()

In [ ]:
test.NumberOfWindows.unique()

In [ ]:
train.Claim.isnull().sum()

In [ ]:
BuildingPainted = pd.get_dummies(test['Building_Painted'], drop_first = True)
BuildingFenced = pd.get_dummies(test['Building_Fenced'], drop_first = True)
garden = pd.get_dummies(test['Garden'], drop_first = True)
settlement = pd.get_dummies(test['Settlement'], drop_first = True)

In [ ]:
BuildingPainted = pd.get_dummies(train['Building_Painted'], drop_first = True)
BuildingFenced = pd.get_dummies(train['Building_Fenced'], drop_first = True)
garden = pd.get_dummies(train['Garden'], drop_first = True)
settlement = pd.get_dummies(train['Settlement'], drop_first = True)

In [ ]:
garden.head()

In [ ]:
settlement.head()

In [ ]:
train.drop(['Customer Id','Geo_Code', 'Settlement','Garden'], axis =1, inplace = True)

In [ ]:
test.drop(['Geo_Code', 'Settlement','Garden'], axis =1, inplace = True)

In [ ]:
test = pd.concat([test, BuildingPainted, BuildingFenced, garden, settlement], axis = 1)

In [ ]:
train = pd.concat([train, BuildingPainted, BuildingFenced, garden, settlement], axis = 1)

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
train.Claim.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
train_imp = pd.DataFrame(my_imputer.fit_transform(train))

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
test_imp = pd.DataFrame(my_imputer.fit_transform(test))

In [ ]:
#Fill in the lines below: Imputation removed column names; put them back
train_imp.columns = train.columns
test_imp.columns = test.columns

In [ ]:
sns.heatmap(test_imp.isnull(), yticklabels = False, cbar= False, cmap = 'viridis')

In [ ]:
test_imp.head()

In [ ]:
test_imp.isna().sum()

In [ ]:
sns.heatmap(train_imp.isnull(), yticklabels = False, cbar= False, cmap = 'viridis')

In [ ]:
train_imp.head()

In [ ]:
train_imp.columns

In [ ]:
train_imp.NumberOfWindows.isna().sum()

In [ ]:
X_test = test_imp.drop(['Customer Id', 'Building_Painted', 'Building_Fenced'], axis = 1)

In [ ]:
X_test = X_test.apply(pd.to_numeric, errors='coerce')

In [ ]:
X = train_imp.drop(['Claim', 'Building_Painted', 'Building_Fenced'], axis = 1)

In [ ]:
X= X.apply(pd.to_numeric, errors='coerce')

In [ ]:
y = train_imp['Claim']

In [ ]:
y = y.apply(pd.to_numeric, errors='coerce')

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=7)

In [ ]:
#import XGBoost classifier and accuracy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#instantiate model and train
model = XGBClassifier(learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
model.fit(X_train, y_train)

# make predictions for test set
y_pred = model.predict(X_val)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
X.head()

In [ ]:
X_test.head()

In [ ]:

test_predictions = model.predict(X_test)
output = pd.DataFrame({'Customer Id': test['Customer Id'],
                      'claim': test_predictions})
output.to_csv('submission.csv', index = False)